Treat image ROI takes electron diffraction patterns and calibrate the image, rotate it, draw the BZ around the Bragg peak and average equivalent BZ together

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.patches import Polygon
from matplotlib.patches import Circle
from mpl_toolkits.axes_grid1 import make_axes_locatable
import imutils
import glob
import cv2  # importing cv
import find_peak
import scipy.special as special
from matplotlib import ticker
from scipy.fft import fft, fftfreq
from scipy.optimize import curve_fit
import nfft
import scipy
import scipy.fft
import functions_analysis as fa
from skimage import img_as_float
from skimage import filters
%matplotlib inline 

path = r'C:\Path\To\Data'
path_main=path+r'\PROCESSED'
scan_number = [ '800nm',  '400nm' ]
scans = [ '1.55 eV',  '3.1 eV' ]


data = {}

for k, scan_name in enumerate(scans):
    data[scan_name] = {}
    data[scan_name] = {}
    df= pd.read_pickle(path_main + r'\PROC_'+ scan_number[k], 
                       compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

    data[scan_name]['imgOFF'] = np.stack(df['imgOFF'])
    data[scan_name]['imgON'] = np.stack(df['imgON'])
    data[scan_name]['delay'] = np.stack(df['delay'])
    data[scan_name]['t0_index'] = df.attrs['t0_index']
    # print(df.attrs['peak_dist'])
    data[scan_name]['Zorder'] = df.attrs['Zorder_pos']
    data[scan_name]['cal'] = df.attrs['calibration']

    data[scan_name]['kx'] = data[scan_name]['cal'] *(np.arange(np.size(data[scan_name]['imgON'][0],1)) -
                                                      data[scan_name]['Zorder'] [1])
    data[scan_name]['ky'] = data[scan_name]['cal'] *(np.arange(np.size(data[scan_name]['imgON'][0],0)) - 
                                                     data[scan_name]['Zorder'] [0])


In [ ]:
L = df.attrs['peak_dist']
Zorder = df.attrs['Zorder_pos']
cal = df.attrs['calibration']
t0 = df.attrs['t0_index']
a= 2.46
b =(4*np.pi/(np.sqrt(3)*a)) 
cal_b = cal/b # for normalized q-vector 


cc =  L * np.sqrt(3)/3 *np.exp(1j * np.pi  * ( 1./3 * np.arange(7))) ## for hexagonal BZ


#### construction position of the Bragg peak, ordered by family
BP_BZ = []
for i in range(6):
    BP_BZ.append([L*np.cos(np.pi*i/3), L*np.sin(np.pi*i/3)])
BP_pos = [Zorder + np.array(BP_BZ)]
BP_BZ = []
for i in range(6):
    BP_BZ.append([2*L*np.cos(np.pi*i/3), 2*L*np.sin(np.pi*i/3)])
BP_pos.append(Zorder + np.array(BP_BZ))
BP_BZ = []
for i in range(6):
    BP_BZ.append([np.sqrt(3)*L*np.cos(np.pi*i/3+np.pi/6), np.sqrt(3)*L*np.sin(np.pi*i/3+np.pi/6)])
BP_pos.append(Zorder + np.array(BP_BZ))
BP_name = ['{100}', '{200}', '{110}']
BP_BZ = []
print(BP_pos)


#### construction position of the K-pont, ordered by family

edge = L*np.tan(np.pi/6)
BP_DS = []
for i in range(6):
    BP_DS.append([2*edge*np.cos(np.pi*i/3+np.pi/6), 2*edge*np.sin(np.pi*i/3+np.pi/6)])
DS_pos = [Zorder + np.array(BP_DS)]
BP_DS = []
for i in range(6):
    BP_DS.append([4*edge*np.cos(np.pi*i/3+np.pi/6), 4*edge*np.sin(np.pi*i/3+np.pi/6)])
DS_pos.append(Zorder + np.array(BP_DS))
BP_DS = []
for i in range(6):  
    BP_DS.append(fa.rotate_vector(np.array([3*L/2, edge/2]), i*np.pi/3))
    BP_DS.append(fa.rotate_vector(np.array([3*L/2,-edge/2]), i*np.pi/3))
DS_pos.append(Zorder + np.array(BP_DS))
BP_DS = []
for i in range(6):
    BP_DS.append(fa.rotate_vector(np.array([2*L, edge]), i*np.pi/3))
    BP_DS.append(fa.rotate_vector(np.array([2*L,-edge]), i*np.pi/3))
DS_pos.append(Zorder + np.array(BP_DS))
BP_DS = []
    # BP_DS.append(fa.rotate_vector(np.array([5*L/2, edge/2]), i*np.pi/3))
    # BP_DS.append(fa.rotate_vector(np.array([5*L/2,-edge/2]), i*np.pi/3))

DS_name = [r'$\Gamma \to K$ first order', r'$\Gamma \to K$ second order', r'$\Gamma \to M$ first order', r'$\Gamma \to M$ second order']

In [ ]:
plt.figure(figsize = (16, 8))

lim=7.5
for k, scan_name in enumerate(scans):

    beforet0 =  data[scan_name]['delay']<-0.5
    aftert0 = ( data[scan_name]['delay']>0.2 ) & ( data[scan_name]['delay']<10)

    diff=(np.mean(data[scan_name]['imgON'][aftert0], axis=0))/np.mean(data[scan_name]['imgOFF'][aftert0], axis=0) - \
        (np.mean(data[scan_name]['imgON'][beforet0], axis=0))/np.mean(data[scan_name]['imgOFF'][beforet0], axis=0)
    ax = plt.subplot(1,2 ,k+1)
    ax.pcolorfast(data[scan_name]['kx'], data[scan_name]['ky'], diff, vmin=-0.01, vmax = 0.01, cmap = 'bwr')
    plt.gca().set_aspect('equal')

    plt.title(scan_name, pad = 20)

    plt.xlim([-lim, lim]), plt.ylim([-lim, lim])
    plt.xticks([-5, 0, 5]), plt.yticks([-5, 0, 5])

    plt.xlabel(r'Momentum (1/\AA)')
    if k ==0:
        plt.ylabel(r'Momentum (1/\AA)')
    else:
        ax.set_yticklabels([])
    ax.tick_params(axis='both', direction='in', pad = 15)
    for i in range(len(BP_pos)):
        for j in range(len(BP_pos[i])):
            ax.add_patch(Circle((cal*(Zorder[1]-BP_pos[i][j][1]), cal*(Zorder[0]-BP_pos[i][j][0])), 0.1, fill = False, edgecolor = 'k', lw = 0.5))
            ax.add_patch(Polygon(cal*np.c_[(cc.real+Zorder[1]-BP_pos[i][j][1]), (cc.imag+Zorder[0]-BP_pos[i][j][0])],
                lw=0.2, edgecolor='k',
                clip_on=False, facecolor='none'))
            
    

In [4]:
plt.rcParams.update({'font.size': 20})
plt.rc('axes', linewidth=2)
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['ytick.minor.width'] = 1
plt.rc('axes', linewidth=2)
plt.rc('font', family='arial',size=26)
plt.rc('text', usetex=False)

In [5]:
for scan_name in scans:
    data[scan_name]['imgON_not_filtered'] = data[scan_name]['imgON']
    data[scan_name]['imgOFF_not_filtered'] = data[scan_name]['imgOFF']

In [6]:
s = 1
for scan_name in scans:
    data[scan_name]['imgON'] = filters.gaussian(img_as_float(data[scan_name]['imgON']), sigma=s)
    data[scan_name]['imgOFF'] = filters.gaussian(img_as_float(data[scan_name]['imgOFF']), sigma=s)

In [ ]:
####################### PLOT FIG 2 A - C ##################################
kx_cal_b = data[scan_name]['kx']/b
ky_cal_b = data[scan_name]['ky']/b


w = .5/b
## parameter for the binning
C = np.array([0,0])
print(BP_pos[0][0])
t1, t2, t3, t4 = 0.5, 2, 4, 8
lim = 240*cal_b
tick = 1
alpha = 0.3
rectangle_ver = np.array([[(C[1]-w),0.9],[(C[1]-w), 1.1],[(C[1]+w),1.1], [(C[1]+w),0.9]])
x=3
y=1



BP_100_name = [r'$\Gamma_{100}$', r'$\Gamma_{1-10}$',r'$\Gamma_{0}$',r'$\Gamma_{100}$',r'$\Gamma_{100}$',r'$\Gamma_{100}$']

plt.figure(figsize = (16, 16))

order = [0,2, 1, 3]
count = 0
sens=0.004
gs = gridspec.GridSpec(2,2, width_ratios=(1,1), wspace = 0.1, hspace = 0.1)

for k, scan_name in enumerate(scans):
    beforet0 =  data[scan_name]['delay']<-0.5
    for kk in [0,1]:
        if kk==0:
            aftert0 = ( data[scan_name]['delay']>t1 ) & ( data[scan_name]['delay']<t2)

        else:
            aftert0 = ( data[scan_name]['delay']>t3 ) & ( data[scan_name]['delay']<t4)

        diff=(np.mean(data[scan_name]['imgON'][aftert0], axis=0))/np.mean(data[scan_name]['imgOFF'][aftert0], axis=0) - \
            (np.mean(data[scan_name]['imgON'][beforet0], axis=0))/np.mean(data[scan_name]['imgOFF'][beforet0], axis=0)
        
        ax = plt.subplot(gs[order[count]])
        ax.pcolorfast(kx_cal_b, ky_cal_b, diff, vmin=-sens, vmax = sens, cmap = 'bwr')
        plt.gca().set_aspect('equal')
        plt.xlim([-lim, lim]), plt.ylim([-lim, lim])
        plt.xticks([-2*tick, -tick, 0, tick, 2*tick]), plt.yticks([-2*tick, -tick, 0, tick, 2*tick])

        if order[count] == 0 or order[count] ==2:
            plt.ylabel(r'$q_y/b$')
        else:
            ax.set_yticklabels([])
        if order[count] >1:
            plt.xlabel(r'$q_x/b$')
        else:
            ax.set_xticklabels([])
            plt.title(scan_name, fontsize=22, pad = 20)
        
        ax.tick_params(axis='both', direction='in', pad = 15)
        count+=1
        for i in range(len(BP_pos)):
            for j in range(len(BP_pos[i])):
                ax.add_patch(Polygon(cal_b*np.c_[(cc.real+Zorder[1]-BP_pos[i][j][1]), (cc.imag+Zorder[0]-BP_pos[i][j][0])],
                    lw=0.2, edgecolor='gray',
                    clip_on=False, facecolor='none'))
        ax.add_patch(Polygon(cal_b*np.c_[(cc.real), (cc.imag)],
                    lw=0.2, edgecolor='gray',
                    clip_on=True, facecolor='white'))
        if k==0 and kk == 0:
            ax.add_patch(Polygon(rectangle_ver, lw=1, linestyle='--', edgecolor='k', fill=False, facecolor='y', alpha=1))
            # t.set_bbox(dict(facecolor='white', alpha=0.8, edgecolor='white'))

        if kk==0 and k==1:
            for i in range(len(BP_pos[0])):
                ax.add_patch(Circle((cal_b*(Zorder[1]-BP_pos[0][i][1]), cal_b*(Zorder[0]-BP_pos[0][i][0])), 0.1, fill = False, edgecolor = 'k', lw = 0.5))

            t = plt.text(cal_b*(Zorder[1]-BP_pos[0][x][1]+5), cal_b*(Zorder[0]-BP_pos[0][x][0]+5), r'$\Gamma_{100}$', c='k', fontsize = 22, ha='left', fontweight = 'bold')
            plt.annotate('',xy=(cal_b*(Zorder[1]-BP_pos[0][y][1]), cal_b*(Zorder[0]-BP_pos[0][y][0])+0.1), xytext=(cal_b*(Zorder[1]-BP_pos[0][y+1][1]), cal_b*(Zorder[0]-BP_pos[0][y+1][0])-0.1), arrowprops=dict(arrowstyle='<->', lw=2))
            t = plt.text(cal_b*(Zorder[1]-BP_pos[0][y][1]-5), cal_b*(Zorder[0]-BP_pos[0][y][0]+40), r'$b$', c='k', fontsize = 22, ha='right', fontweight = 'bold')


        if kk==0:
            aftert0 = ( data[scan_name]['delay']>0.1 ) & ( data[scan_name]['delay']<2)
            text_time = r'0.5 ps $<$\textit{t}$<$ 2 ps'
            text_time = r'%.1f ps $< \Delta t <$ %d ps'%(t1,t2)
            t = plt.text(220*cal_b, 200*cal_b, text_time, c='k', fontsize = 22, ha='right')
            t.set_bbox(dict(facecolor='white', alpha=0.8, edgecolor='white'))


        else:
            aftert0 = ( data[scan_name]['delay']>2 ) & ( data[scan_name]['delay']<8)
            text_time = r'2 ps $<\textit{t}<$ 8 ps'
            text_time = r'%d ps $< \Delta t <$ %d ps'%(t3,t4)
            t = plt.text(220*cal_b, 200*cal_b, text_time, c='k', fontsize = 22, ha='right')
            t.set_bbox(dict(facecolor='white', alpha=0.8, edgecolor='white'))


    

In [ ]:
####################### PLOT FIG 2 D - F ##################################
plt.figure(figsize = (2, 16))
c=['r', 'b']
w = 0.5
sens=0.002
lim=15
dk=0.1*b 
print(t3, t4)
for k, scan_name in enumerate(scans):
    beforet0 =  data[scan_name]['delay']<-0.5
    aftert0 = ( data[scan_name]['delay']>t3 ) & ( data[scan_name]['delay']<t4)
    kx_cut = (data[scan_name]['kx']>-w) & (data[scan_name]['kx']<w)

    ## ON OFF
    GM_dynamics = (np.mean(data[scan_name]['imgON'][:, :, kx_cut], axis=2))/(np.mean(data[scan_name]['imgOFF'][:,:, kx_cut], axis=2))
    GM_late = (np.mean(np.mean(data[scan_name]['imgON'][:, :, kx_cut], axis=2)[aftert0,:], axis=0))/(np.mean(np.mean(data[scan_name]['imgOFF'][:, :, kx_cut], axis=2)[aftert0,:], axis=0))
    GM_bt0 = (np.mean(np.mean(data[scan_name]['imgON'][:, :, kx_cut], axis=2)[beforet0,:], axis=0))/(np.mean(np.mean(data[scan_name]['imgOFF'][:, :, kx_cut], axis=2)[beforet0,:], axis=0))

    ax = plt.subplot(3,1, k+1)
    ax.pcolor(data[scan_name]['ky'], data[scan_name]['delay'], (GM_dynamics- np.mean(GM_dynamics[beforet0], axis=0)), vmin=-sens, vmax = sens, cmap = 'bwr')
    ax.set_xlim([L*cal-dk, L*cal+dk])
    ax.set_ylim([-1, 8])
    ax.set_ylabel('Time delay (ps)')
    ax.tick_params(axis='both', direction='in', pad = 15)

    t = plt.text(L*cal-dk*0.9, -0.5, scan_name, c='k', fontsize = 26)
    t.set_bbox(dict(facecolor='white', alpha=0.8, edgecolor='white'))

    ax.set_xticks([L*cal-dk, L*cal ,L*cal+dk])
    ax.set_xticklabels([])
    ax.tick_params(axis='both', direction='in', pad = 15)
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position('right')



    ax2 = plt.subplot(3,1,3)
    ax2.yaxis.tick_right()
    ax2.yaxis.set_label_position('right')
    ax2.plot(data[scan_name]['ky'],1e3*( GM_late-GM_bt0)+k*1.5, c=c[k], linewidth = 4, label=scan_name)
    ax2.plot(data[scan_name]['ky'],1e3*(GM_dynamics[1]- GM_bt0)+k*1.5, '--', c=c[k], linewidth = 4)
    ax2.set_xlim([L*cal-dk, L*cal+dk])
    ax2.set_ylim([-1,3.5])
    ax2.set_xticks([L*cal-dk, L*cal ,L*cal+dk])
    ax2.set_xticklabels([0.9,1,1.1])
    ax2.tick_params(axis='both', direction='in', pad = 15)


ax2.set_ylabel(r'intensity variation (‰)')
ax2.set_xlabel(r'$q_y/b$ ( )')
ax2.legend(loc='upper left')

plt.subplots_adjust(hspace = 0.1)